In [40]:
import numpy as np
import random
import time
import pandas as pd
import os

In [63]:
def extract_display_size(input_string):
    try:
        split_values = input_string.split('s, ')
        screen_size_inch = split_values[0].split(' ')[0]
    except:
        screen_size_inch = np.nan

    try:
        screen_area_cm = split_values[1].split(' ')[0]
    except:
        screen_area_cm = np.nan
    try:
        screen_to_body_ratio = split_values[1].split('~')[1].split('%')[0]  # Extract screen-to-body ratio
    except:
        screen_to_body_ratio = np.nan

    return screen_size_inch, screen_area_cm, screen_to_body_ratio

def extract_display_resolution(input_string):
    try:
        split_values = input_string.split('s, ')
        pixels = split_values[0].split(' ')[0]
    except:
        pixels = np.nan

    try:
        ratio = split_values[1].split(' ')[0]
    except:
        # print(input_string)
        ratio = np.nan
    try:
        ppi_density = split_values[1].split('~')[1].split(' ')[0]  # Extract screen-to-body ratio
    except:
        ppi_density = np.nan

    return pixels, ratio, ppi_density

In [80]:
df = pd.read_csv('Step1_df.csv')
df['Device_label'].unique()

array(['phone', 'tablet', 'watch'], dtype=object)

In [82]:
selected_columns = ['Display_Size', 'Display_Resolution', 'Display_Type', 'Platform_OS', 'Platform_CPU', 'Memory_Card slot', 'Memory_Internal']

df = pd.read_csv('Step1_df.csv')

df_phone = df.loc[df['Device_label'] == 'phone',:]
df_tablet = df.loc[df['Device_label'] == 'tablet',:]
df_watch = df.loc[df['Device_label'] == 'watch',:]

df_phone = df_phone.loc[:, selected_columns]
df_tablet = df_tablet.loc[:, selected_columns]
df_watch = df_watch.loc[:, selected_columns]

df['Display_Size_inches'], df['Display_Size_cm2'], df['screen_to_body_ratio'] = zip(*df['Display_Size'].apply(extract_display_size))
df['Display_Resolution_pixels'], df['Display_Resolution_ratio'], df['Display_Resolution_ppi'] = zip(*df['Display_Resolution'].apply(extract_display_resolution))
df['Display_Type'] = df['Display_Type'].apply(lambda col: col.split(',')[0])
df['Platform_OS'], df['Platform_OS_version'] = df['Platform_OS'].apply(lambda col: col.split(',')[0].split(' '))



df.drop(columns=['Display_Size', 'Display_Resolution'], inplace=True)

AttributeError: 'float' object has no attribute 'split'

In [83]:
df_phone['Platform_OS'].unique()

array(['Android 11 (Go edition)', 'Android 11', 'Android 10', 'KaiOS 3.0',
       'KaiOS 2.5.2', 'Android 10 (Go edition)', 'Android 9.0 (Pie)',
       'Android 9.0 Pie (Go edition)', 'Android 8.1 (Oreo)',
       'Android 8.1 Oreo (Go edition)', 'Android 7.1 (Nougat)',
       'Android 8.0 (Oreo)',
       'Android 8.1 (Oreo) - 5059D only, Android 8.1 Oreo (Go edition) - all other models',
       'KaiOS 2.5.1', 'Android 7.0 (Nougat)', 'Android 6.0 (Marshmallow)',
       'Android 5.1 (Lollipop)', 'Microsoft Windows 10',
       'Android 6.0.1 (Marshmallow)',
       'Android 5.1 (Lollipop), upgradable to 6.0.1 (Marshmallow)',
       'Android 5.1.1 (Lollipop)', nan, 'Android 5.0 (Lollipop)',
       'Android 4.4.2 (KitKat)', 'Android 4.4.4 (KitKat)',
       'Android 4.4 (KitKat)',
       'Android 5.0.2 (Lollipop), upgradable to 6.0.1 (Marshmallow)',
       'Firefox 2.0',
       'Android 4.4.2 (KitKat) / Android 5.0 (Lollipop) - 4G model',
       'Android 4.2.2 (Jelly Bean), upgradable to 5.0 

In [77]:
df['Platform_OS'].unique()

array(['Android 11 (Go edition)', 'Android 11', 'Android 10', 'KaiOS 3.0',
       'KaiOS 2.5.2', 'Android 10 (Go edition)', 'Android 9.0 (Pie)',
       'Android 9.0 Pie (Go edition)', 'Android 8.1 (Oreo)',
       'Android 8.1 (Oreo), upgradable to Android 9.0 (Pie)',
       'Android 8.1 Oreo (Go edition)', 'Android 7.1 (Nougat)',
       'Android 8.0 (Oreo)',
       'Android 8.1 (Oreo) - 5059D only, Android 8.1 Oreo (Go edition) - all other models',
       'KaiOS 2.5.1', 'Android 7.0 (Nougat)', 'Android 6.0 (Marshmallow)',
       'Android 5.1 (Lollipop)', 'Android 6.0.1 (Marshmallow)',
       'Microsoft Windows 10', 'Proprietary OS',
       'Android 5.1 (Lollipop), upgradable to 6.0.1 (Marshmallow)',
       'Android 5.1.1 (Lollipop)', nan, 'Android 5.0 (Lollipop)',
       'Android 4.4.2 (KitKat)', 'Android 4.4.4 (KitKat)',
       'Android 4.4 (KitKat)',
       'Android 5.0.2 (Lollipop), upgradable to 6.0.1 (Marshmallow)',
       'Firefox 2.0', 'Android 5.0.1 (Lollipop)',
       'Android